**MLOps**

**Usando BentoML para publicação de modelos em produção**

Exemplo de uso do BentoML para publicação de modelos em produção.

Link: https://colab.research.google.com/github/bentoml/gallery/blob/master/fasttext/text-classification/text-classification.ipynb#scrollTo=FIBXoXd-_dOw

In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

import warnings
warnings.filterwarnings("ignore")

In [28]:
# instalando fasttext by Facebook
!pip install -q fasttext==0.9.2

In [4]:
# instalando bentoml    
!pip install -q bentoml "fasttext==0.9.2"

## **Preparando os Dados**

In [2]:
# baixa o dataset
!curl https://dl.fbaipublicfiles.com/fasttext/data/cooking.stackexchange.tar.gz --output cooking.stackexchange.tar.gz && tar xvzf cooking.stackexchange.tar.gz

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  446k  100  446k    0     0   199k      0  0:00:02  0:00:02 --:--:--  199k
x cooking.stackexchange.id
x cooking.stackexchange.txt
x readme.txt


In [6]:
!head cooking.stackexchange.txt

__label__sauce __label__cheese How much does potato starch affect a cheese sauce recipe?
__label__food-safety __label__acidity Dangerous pathogens capable of growing in acidic environments
__label__cast-iron __label__stove How do I cover up the white spots on my cast iron stove?
__label__restaurant Michelin Three Star Restaurant; but if the chef is not there
__label__knife-skills __label__dicing Without knife skills, how can I quickly and accurately dice vegetables?
__label__storage-method __label__equipment __label__bread What's the purpose of a bread box?
__label__baking __label__food-safety __label__substitutions __label__peanuts how to seperate peanut oil from roasted peanuts at home?
__label__chocolate American equivalent for British chocolate terms
__label__baking __label__oven __label__convection Fan bake vs bake
__label__sauce __label__storage-lifetime __label__acidity __label__mayonnaise Regulation and balancing of readymade packed mayonnaise and other sauces


In [3]:
# Defining train and valitation datasets
!head -n 12404 cooking.stackexchange.txt > cooking.train
!head -n 3000 cooking.stackexchange.txt > cooking.valid

## **Train Madel**

In [4]:
import fasttext

In [7]:
model = fasttext.train_supervised(input="cooking.train")

## **Define and Save BentoService**

In [8]:
# gera o arquivo de implementacao do serviço bentoml
%%writefile text_classification.py

from bentoml import env, artifacts, BentoService, api
from bentoml.frameworks.fasttext import FasttextModelArtifact
from bentoml.adapters import JsonInput

@env(infer_pip_packages=True)
@artifacts([FasttextModelArtifact('model')])
class FasttextClassification(BentoService):

  @api(input=JsonInput(), batch=True)
  def predict (self, json_list):
    input = [i['text'] for i in json_list]
    result = self.artifacts.model.predict(input)
    # return top result
    prediction_result = [i[0].replace('__label__','') for i in result[0]]
    return prediction_result

Writing text_classification.py


In [10]:
from text_classification import FasttextClassification

svc = FasttextClassification()
svc.pack('model', model)

saved_path = svc.save()

[2021-04-19 21:10:22,475] INFO - BentoService bundle 'FasttextClassification:20210419211021_3A6C2C' saved to: /Users/roberto/bentoml/repository/FasttextClassification/20210419211021_3A6C2C


In [11]:
saved_path

'/Users/roberto/bentoml/repository/FasttextClassification/20210419211021_3A6C2C'

In [41]:
!ls -l /root/bentoml/repository/

total 4
drwxr-xr-x 5 root root 4096 Apr 19 23:36 FasttextClassification


## **REST API Model Serving**

In [ ]:
!bentoml serve {saved_path}

[2021-04-19 21:11:02,853] INFO - Starting BentoML API proxy in development mode..
[2021-04-19 21:11:02,857] INFO - Starting BentoML API server in development mode..
[2021-04-19 21:11:04,067] INFO - Your system nofile limit is 4096, which means each instance of microbatch service is able to hold this number of connections at same time. You can increase the number of file descriptors for the server process, or launch more microbatch instances to accept more concurrent connection.
======== Running on http://0.0.0.0:5000 ========
(Press CTRL+C to quit)
 * Serving Flask app "FasttextClassification" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off
 * Running on http://127.0.0.1:61138/ (Press CTRL+C to quit)
127.0.0.1 - - [19/Apr/2021 21:13:02] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [19/Apr/2021 21:13:02] "GET /static_content/main.css HTTP/1.1" 200 -
127.0.0.1 - - [19/Apr/2021 21:13:02] "GET /static_content/readme.css HTTP/1.1" 200 -
127.0.0.

In [21]:
!bentoml serve FasttextClassification:latest --run-with-ngrok

/usr/local/lib/python3.7/dist-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)
[2021-04-19 20:05:57,271] INFO - Getting latest version FasttextClassification:20210419194348_810FF3
[2021-04-19 20:05:57,288] INFO - Starting BentoML API proxy in development mode..
[2021-04-19 20:05:57,290] INFO - Starting BentoML API server in development mode..
[2021-04-19 20:05:57,418] INFO - Your system nofile limit is 1048576, which means each instance of microbatch service is able to hold this number of connections at same time. You can increase the number of file descriptors for the server process, or launch more microbatch instances to accept more concurrent connection.
======== Running on http://0.0.0.0:5000 ========
(Press CTRL+C to quit)
 * Serving Flask 

In [ ]:
curl -X POST \
  http://localhost:5000/predict \
  -H 'Content-Type: application/json' \
  -d '{"text": "Which baking dish is best to bake a banana bread ?"}'

In [ ]:
curl -X POST \
  http://localhost:5000/predict \
  -H 'Content-Type: application/json' \
  -d '{"text": "purpose of a bread box ?"}'

## **Containerize model server with Docker**

In [ ]:
!bentoml containerize FasttextClassification:latest
# Ao final do build -> fasttextclassification:20210419211021_3A6C2C 

In [ ]:
# observar ao final da etapa de coteinerização
!docker run --rm -p5000:5000 fasttextclassification:20210419211021_3A6C2C

## **Load Saved BentoService**

In [51]:
from bentoml import load

svc = load(saved_path)

print(svc.predict([{"text":"which baking dish is the best?"}]))

[2021-04-19 23:44:33,833] WARNING - Module `text_classification` already loaded, using existing imported module.
[2021-04-19 23:44:33,846] WARNING - pip package requirement fasttext already exist


NameError: ignored

##Launch Inference Job from CLI

In [52]:
!bentoml run FasttextClassification:latest predict --input '{"text":"Which baking dish is best to bake a banana bread ?"}'

/usr/local/lib/python3.7/dist-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)
[2021-04-19 23:45:04,808] INFO - Getting latest version FasttextClassification:20210419233630_E6A85A
2021-04-19 23:45:06.428968: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
[2021-04-19 23:45:07,739] INFO - {'service_name': 'FasttextClassification', 'service_version': '20210419233630_E6A85A', 'api': 'predict', 'task': {'data': '{"text":"Which baking dish is best to bake a banana bread ?"}', 'task_id': '9b9b535a-fd36-4123-8504-f451dd416ec6', 'cli_args': ('--input', '{"text":"Which baking dish is best to bake a banana bread ?"}'), 'inference_job_args': {}}, 'result': {'data': '"baking"', 'http_stat